# Training the Algorithmic Traders

This notebook contains the code for generating and training the LSTM traders. We will be using the keras structures that are available with tensorflow as the basis for the struture.

In [120]:
import numpy as np #Import relevant packages
import pandas as pd
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
import csv

In [171]:
def unpack_data(filepath): #Returns a dictionary where each item is a df containing one run. One dict per generation.
    all_files = glob.glob(filepath + "/*.csv")
#    print(all_files)
    name_list = []
    datadict = {}
    for f in all_files:
        name_list.append(f[77:-4])
#        print(name_list)
    for i,n in enumerate(name_list):
        datadict[n] = pd.read_csv(all_files[i], header=0)
    for k, v in datadict.items():
        v.drop("Unnamed: 0", axis=1, inplace=True)
#        v.reset_index(drop=True)
#        v.index.name = "period"
    return datadict

In [172]:
gen0 = unpack_data("/Users/karangarg/Documents/Year 3 Modules/EC331/Code/rae_repo/gen0_sims/data") #Load gen0 data

In [175]:
totaldf = pd.DataFrame(columns=["trading_price", "volume", "spread", "true_price", "10_MA", "50_MA"])
    
for k, v in gen0.items(): #Create a dataframe that contains all gen 0 data
    pd.concat([totaldf, v], sort=False)

totaldf

,trading_price,volume,spread,true_price,10_MA,50_MA


In [128]:
scaler = MinMaxScaler()

scaler.fit()

In [173]:
gen0["gen0sim1"]

,trading_price,volume,spread,true_price,10_MA,50_MA
0,100.00,0.0,0.00,100.00,99.37,99.38
1,99.70,93.0,1.92,100.10,99.19,99.45
2,99.24,27.0,1.85,100.20,99.19,99.48
3,99.05,59.0,1.85,100.30,99.14,99.52
4,99.27,112.0,1.92,100.40,99.12,99.59
5,99.24,99.0,1.88,100.50,99.12,99.65
6,99.06,73.0,1.85,100.60,99.27,99.69
7,98.76,33.0,1.81,100.70,99.22,99.73
8,98.21,29.0,1.81,100.78,99.16,99.77
9,98.32,139.0,1.89,100.38,98.93,99.82
